In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms as T
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import timm
from torch import nn


In [3]:
TRAIN_IMG_FOLDER_PATH = r"D:\emotion detection\archive\dataset\train0"
VALID_IMG_FOLDER_PATH = r"D:\emotion detection\archive\dataset\validation0"

In [4]:
LR = 0.001
BATCH_SIZE = 32
EPOCHS = 15
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_NAME = 'efficientnet_b0'
NUM_CLASSES = 7  # Number of classes


In [5]:
# Load Dataset
train_augs = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=(-20, +20)),
    T.ToTensor()
])


In [6]:
valid_augs = T.Compose([
    T.ToTensor()
])


In [7]:
trainset = ImageFolder(TRAIN_IMG_FOLDER_PATH, transform=train_augs)
validset = ImageFolder(VALID_IMG_FOLDER_PATH, transform=valid_augs)


In [8]:
print(f"Total no. of examples in trainset: {len(trainset)}")
print(f"Total no. of examples in validset: {len(validset)}")


Total no. of examples in trainset: 315
Total no. of examples in validset: 315


In [9]:
class_names = trainset.classes
print(class_names)


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [10]:
# Split trainset into train and validation
train_indices, val_indices = train_test_split(list(range(len(trainset))), test_size=0.2, random_state=42)
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)


In [11]:
# Create DataLoaders
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, sampler=train_sampler)
validloader = DataLoader(validset, batch_size=BATCH_SIZE, sampler=val_sampler)

print(f"Total no. of batches in trainloader: {len(trainloader)}")
print(f"Total no. of batches in validloader: {len(validloader)}")


Total no. of batches in trainloader: 8
Total no. of batches in validloader: 2


In [12]:
# Create Model
class DepressionModel(nn.Module):
    def __init__(self, num_classes):
        super(DepressionModel, self).__init__()
        self.eff_net = timm.create_model(MODEL_NAME, pretrained=True, num_classes=num_classes)

    def forward(self, images, labels=None):
        logits = self.eff_net(images)
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            return logits, loss
        return logits

model = DepressionModel(num_classes=NUM_CLASSES).to(DEVICE)


In [13]:
# Define training and evaluation functions
def train_fn(model, dataloader, optimizer, current_epo):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    tk = tqdm(dataloader, desc=f"EPOCH [TRAIN] {current_epo + 1}/{EPOCHS}")

    for t, data in enumerate(tk):
        images, labels = data
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        logits, loss = model(images, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = logits.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

        tk.set_postfix({'loss': f'{total_loss / (t + 1):.6f}', 'accuracy': f'{correct / total:.6f}'})

    return total_loss / len(dataloader), correct / total


In [14]:
def eval_fn(model, dataloader, current_epo):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    tk = tqdm(dataloader, desc=f"EPOCH [VALID] {current_epo + 1}/{EPOCHS}")

    with torch.no_grad():
        for t, data in enumerate(tk):
            images, labels = data
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            logits, loss = model(images, labels)

            total_loss += loss.item()
            _, predicted = logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            tk.set_postfix({'loss': f'{total_loss / (t + 1):.6f}', 'accuracy': f'{correct / total:.6f}'})

    return total_loss / len(dataloader), correct / total


In [15]:
# Create optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LR)

best_valid_loss = np.Inf

In [16]:
# Training loop
for epoch in range(EPOCHS):
    train_loss, train_acc = train_fn(model, trainloader, optimizer, epoch)
    valid_loss, valid_acc = eval_fn(model, validloader, epoch)

    if valid_loss < best_valid_loss:
        torch.save(model.state_dict(), 'best_weights.pt')
        print("Saved best weights")
        best_valid_loss = valid_loss


EPOCH [VALID] 1/15: 100%|██████████| 2/2 [00:00<00:00, 14.23it/s, loss=5.675611, accuracy=0.238095]


Saved best weights


EPOCH [VALID] 6/15: 100%|██████████| 2/2 [00:00<00:00, 17.77it/s, loss=5.546233, accuracy=0.158730]


Saved best weights


EPOCH [VALID] 8/15: 100%|██████████| 2/2 [00:00<00:00, 17.82it/s, loss=5.192978, accuracy=0.190476]


Saved best weights


EPOCH [VALID] 15/15: 100%|██████████| 2/2 [00:00<00:00, 16.92it/s, loss=5.110965, accuracy=0.269841]

Saved best weights


In [17]:
# Load best weights
model.load_state_dict(torch.load('best_weights.pt'))


<All keys matched successfully>

In [18]:
# Inference
def predict(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            logits = model(images)
            _, predicted = logits.max(1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels


In [19]:
# Inference
def predict(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            logits = model(images)
            _, predicted = logits.max(1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels


In [20]:
# Make predictions on validation set
val_predictions, val_true_labels = predict(model, validloader)

# Generate classification report
print(classification_report(val_true_labels, val_predictions))


              precision    recall  f1-score   support

           0       0.30      0.33      0.32         9
           1       0.25      0.09      0.13        11
           2       0.11      0.12      0.12         8
           3       0.33      0.22      0.27         9
           4       0.30      0.33      0.32         9
           5       0.20      0.40      0.27         5
           6       0.36      0.42      0.38        12

    accuracy                           0.27        63
   macro avg       0.26      0.27      0.26        63
weighted avg       0.27      0.27      0.26        63



In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from sklearn.metrics import classification_report

# Import the necessary components for the model
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms as T
import timm
from torch import nn

# Function to preprocess the test image
def preprocess_image(file_path):
    # Load the image
    image = Image.open(file_path)
    
    # Convert to grayscale
    image_gray = image.convert('L')
    
    # Resize to (48, 48)
    image_resized = image_gray.resize((48, 48))
    
    # Convert to RGB
    image_rgb = Image.new("RGB", image_resized.size)
    image_rgb.paste(image_resized)
    
    # Convert to numpy array
    image_np = np.array(image_rgb)
    
    # Convert to torch tensor and normalize
    image_tensor = torch.tensor(image_np, dtype=torch.float32).permute(2, 0, 1) / 255.0
    
    # Expand dimensions to make it a batch of size 1
    image_tensor = image_tensor.unsqueeze(0)
    
    return image_tensor


# Function to make predictions
def predict_image(model, image_tensor):
    model.eval()
    with torch.no_grad():
        logits = model(image_tensor)
        _, predicted = torch.max(logits, 1)
    return predicted.item()
    
# Function to handle button click event
def open_test_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Preprocess the image
        image_tensor = preprocess_image(file_path)
        
        # Display the image
        display_image(file_path)
        
        # Make prediction
        predicted_class = predict_image(model, image_tensor)
        
        # Display result on Tkinter window
        result_label.config(text=f"Predicted Class: {class_names[predicted_class]}")

# Function to display the selected image on canvas
def display_image(file_path):
    # Open the image
    image = Image.open(file_path)
    
    # Resize the image to fit canvas
    image.thumbnail((200, 200))
    
    # Convert the image to PhotoImage format
    photo = ImageTk.PhotoImage(image)
    
    # Display the image on canvas
    canvas.create_image(0, 0, anchor=tk.NW, image=photo)
    canvas.image = photo  # Keep a reference to avoid garbage collection

# Load the trained model and class names
MODEL_NAME = 'efficientnet_b0'
NUM_CLASSES = 7
model = DepressionModel(num_classes=NUM_CLASSES)
model.load_state_dict(torch.load('best_weights.pt'))
model.eval()

# Load class names
class_names = trainset.classes

# Create Tkinter window
root = tk.Tk()
root.title("Emotion Detection")
root.geometry("600x400")

# Configure the background color to black
root.configure(bg="black")


# Create button to select test image
button = tk.Button(root, text="Select Test Image", command=open_test_image, bg="green", fg="white")
button.pack(pady=20)

# Canvas to display the selected image
canvas = tk.Canvas(root, width=200, height=200, bg="black", highlightthickness=0)
canvas.pack()

# Label to display result
result_label = tk.Label(root, text="", bg="black", fg="White")
result_label.pack()

root.mainloop()
